<a href="https://colab.research.google.com/github/Yashvishe-1327/mlc-sp25-aquafina/blob/bird_clef_yash/Birdclef_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
birdclef_2025_path = kagglehub.competition_download('birdclef-2025')
midcarryhz_efficientnet_b0_birdclef_finetuned_path = kagglehub.dataset_download('midcarryhz/efficientnet-b0-birdclef-finetuned')
midcarryhz_efficientnet_b0_test_4_pytorch_default_1_path = kagglehub.model_download('midcarryhz/efficientnet-b0-test_4/PyTorch/default/1')

print('Data source import complete.')

In [ ]:
import time
START = time.time()

from concurrent.futures import ThreadPoolExecutor
import glob
import librosa
import numpy as np
import os
import pandas as pd
import re
import sys
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import torchaudio
import torchaudio.transforms as AT
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy import signal
import gc
import warnings
import logging
from pathlib import Path
import random

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

os.environ['CUDA_VISIBLE_DEVICES'] = ''
torch.set_num_threads(4)

TERMINATE_TIME = START + 5300

class CFG:

    test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
    train_soundscapes = '/kaggle/input/birdclef-2025/train_soundscapes'
    submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
    taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'

    model_path_list = [
        '/kaggle/input/efficientnet-b0-Test-4_stage/model_fold0.pth',
        '/kaggle/input/efficientnet-b0-test-4_stage/pytorch/default/1'
    ]
    model_path = model_path_list[0]

    debug = False
    debug_start_num = 0
    debug_num = 8

    FS = 32000
    WINDOW_SIZE = 5

    N_FFT = 1024
    HOP_LENGTH = 16
    N_MELS = 148
    FMIN = 20
    FMAX = 16000
    TARGET_SHAPE = (256, 256)

    model_name = 'efficientnet_b0'
    in_channels = 1
    device = 'cpu'

    batch_size = 16
    use_tta = False
    tta_count = 3
    threshold = 0.5

    use_specific_folds = False
    folds = [0, 1]

    apply_smoothing = True

    apply_noise_reduction = True
    apply_normalization = True
    noise_reduction_strength = 0.1

    smoothing_window = 5
    smoothing_weights = [0.15, 0.2, 0.3, 0.2, 0.15]

    clear_cache_frequency = 5

    use_spec_augment = True

    time_mask_param = 30
    freq_mask_param = 20
    time_mask_count = 1
    freq_mask_count = 1

    apply_spec_contrast = True
    contrast_factor = 0.15

    class_thresholds = None

    prediction_blend = [0.7, 0.3]

    apply_secondary_smoothing = True
    secondary_smoothing_weights = [0.2, 0.6, 0.2]

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

# PyTorch鸟类分类模型
class BirdCLEFModel(nn.Module):
    def __init__(self, cfg, num_classes):
        super().__init__()
        self.cfg = cfg
        self.backbone = timm.create_model(
            cfg.model_name,
            pretrained=False,
            in_chans=cfg.in_channels,
            drop_rate=0.0,
            drop_path_rate=0.0
        )

        if 'efficientnet' in cfg.model_name:
            backbone_out = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
        elif 'resnet' in cfg.model_name:
            backbone_out = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
        else:
            backbone_out = self.backbone.get_classifier().in_features
            self.backbone.reset_classifier(0, '')

        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.feat_dim = backbone_out
        self.classifier = nn.Sequential(
            nn.Linear(backbone_out, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.15),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.backbone(x)
        if isinstance(features, dict):
            features = features['features']
        if len(features.shape) == 4:
            features = self.pooling(features)
            features = features.view(features.size(0), -1)

        logits = self.classifier(features)
        return logits

def clear_memory():
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

print("Loading label data...")
primary_labels = pd.read_csv('/kaggle/input/birdclef-2025/sample_submission.csv').columns[1:].to_list()
taxonomy = pd.read_csv(CFG.taxonomy_csv)

def estimate_class_thresholds():
    print("Estimating optimal thresholds for each class...")
    default_threshold = CFG.threshold

    thresholds = {}

    if taxonomy is not None and 'primary_label' in taxonomy.columns:
        for species in primary_labels:
            species_info = taxonomy[taxonomy['primary_label'] == species]

            if not species_info.empty:
                family = species_info['family'].iloc[0] if 'family' in species_info.columns else None
                genus = species_info['genus'].iloc[0] if 'genus' in species_info.columns else None

                if family in ['Trochilidae', 'Tyrannidae']:
                    thresholds[species] = default_threshold - 0.05
                elif family in ['Thraupidae', 'Parulidae']:
                    thresholds[species] = default_threshold + 0.05
                else:
                    thresholds[species] = default_threshold
            else:
                thresholds[species] = default_threshold
    else:
        for species in primary_labels:
            thresholds[species] = default_threshold

    return thresholds

def get_audio_files():
    if os.path.exists(CFG.test_soundscapes) and len(glob.glob(f'{CFG.test_soundscapes}/*.ogg')) > 0:
        CFG.debug = False
        audio_dir = CFG.test_soundscapes
        audio_paths = sorted(glob.glob(f'{audio_dir}/*.ogg'))
    else:
        print("No test files found. Using train soundscapes for debugging.")
        CFG.debug = True
        audio_dir = CFG.train_soundscapes
        all_audio_paths = sorted(glob.glob(f'{audio_dir}/*.ogg'))
        audio_paths = all_audio_paths[CFG.debug_start_num:CFG.debug_start_num + CFG.debug_num]

    file_ids = [os.path.splitext(os.path.basename(path))[0] for path in audio_paths]

    print(f'Debug mode: {CFG.debug}')
    print(f'Number of soundscapes: {len(audio_paths)}')

    return audio_paths, file_ids

def find_model_files():
    model_files = []
    for model_path in CFG.model_path_list:
        model_dir = Path(model_path)

        if model_dir.is_file() and model_dir.suffix in ['.pth', '.pt', '.bin']:
            model_files.append(str(model_dir))
        else:
            print(f"Looking for model files in {model_dir}...")

            for ext in ['*.pth', '*.pt', '*.bin']:
                found_files = list(model_dir.glob(f'**/{ext}'))
                if found_files:
                    model_files.extend([str(f) for f in found_files])
            if not model_files and model_dir.exists():
                all_files = list(model_dir.glob('*'))
                for f in all_files:
                    if f.is_file():
                        print(f"Found potential model file: {f}")
                        model_files.append(str(f))

    if model_files:
        print(f"Found {len(model_files)} model files: {model_files}")
    else:
        print(f"No model files found in {CFG.model_path_list}")

    return model_files

def load_models():
    models = []
    model_files = find_model_files()
    if not model_files:
        print(f"Warning: No model files found under {CFG.model_path_list}!")
        return models

    print(f"Found a total of {len(model_files)} model files.")

    if CFG.use_specific_folds:
        filtered_files = []
        for fold in CFG.folds:
            fold_files = [f for f in model_files if f"fold{fold}" in f]
            filtered_files.extend(fold_files)
        model_files = filtered_files
        print(f"Using {len(model_files)} model files for the specified folds ({CFG.folds}).")

    for model_path in model_files:
        try:
            print(f"Loading model: {model_path}")
            checkpoint = torch.load(model_path, map_location='cpu')
            model = BirdCLEFModel(CFG, len(primary_labels))

            if isinstance(checkpoint, dict):
                try:
                    if 'model' in checkpoint:
                        model.load_state_dict(checkpoint['model'], strict=False)
                        model = model.to(CFG.device)
                        model.eval()
                        models.append(model)
                        print(f"Successfully loaded model from {model_path} (non-strict mode)")
                    elif 'model_state_dict' in checkpoint:
                        model.load_state_dict(checkpoint['model_state_dict'], strict=False)
                        model = model.to(CFG.device)
                        model.eval()
                        models.append(model)
                        print(f"Successfully loaded model from {model_path} (non-strict mode)")
                    elif 'state_dict' in checkpoint:
                        model.load_state_dict(checkpoint['state_dict'], strict=False)
                        model = model.to(CFG.device)
                        model.eval()
                        models.append(model)
                        print(f"Successfully loaded model from {model_path} (non-strict mode)")
                    else:
                        print(f"No standard keys found. Trying non-strict direct load.")
                        model.load_state_dict(checkpoint, strict=False)
                        model = model.to(CFG.device)
                        model.eval()
                        models.append(model)
                        print(f"Successfully loaded model from {model_path} (non-strict direct load)")
                except Exception as e:
                    print(f"Error with non-strict loading: {e}")
                    try:
                        if 'model' in checkpoint:
                            new_state_dict = {}
                            for k, v in checkpoint['model'].items():
                                if k.startswith('module.'):
                                    new_state_dict[k[7:]] = v
                                else:
                                    new_state_dict[k] = v
                            model.load_state_dict(new_state_dict, strict=False)
                            model = model.to(CFG.device)
                            model.eval()
                            models.append(model)
                            print(f"Successfully loaded model after DataParallel handling")
                        else:
                            print("No 'model' key found for DataParallel handling")
                    except Exception as e2:
                        print(f"Error with DataParallel handling: {e2}")
            else:
                print(f"Checkpoint is not a dict. Trying non-strict direct load.")
                try:
                    model.load_state_dict(checkpoint, strict=False)
                    model = model.to(CFG.device)
                    model.eval()
                    models.append(model)
                    print(f"Successfully loaded model from {model_path} (non-strict direct load)")
                except Exception as e:
                    print(f"Error loading non-dict checkpoint: {e}")

        except Exception as e:
            print(f"Error loading model {model_path}: {e}")
            import traceback
            traceback.print_exc()

    return models

def reduce_noise(audio_data):
    if not CFG.apply_noise_reduction:
        return audio_data

    window_size = 5
    audio_denoised = signal.medfilt(audio_data, window_size)

    return (1 - CFG.noise_reduction_strength) * audio_data + CFG.noise_reduction_strength * audio_denoised

def normalize_audio(audio_data):
    if not CFG.apply_normalization:
        return audio_data

    audio_data = audio_data - np.mean(audio_data)

    max_amplitude = np.max(np.abs(audio_data))
    if max_amplitude > 0:
        audio_data = audio_data / max_amplitude

    return audio_data

def audio2melspec(audio_data, cfg=CFG):

    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    if len(audio_data) < CFG.FS * CFG.WINDOW_SIZE:
        audio_data = np.pad(
            audio_data,
            (0, CFG.FS * CFG.WINDOW_SIZE - len(audio_data)),
            mode='constant'
        )

    audio_data = reduce_noise(audio_data)
    audio_data = normalize_audio(audio_data)

    mel_spec = librosa.feature.melspectrogram(
        y=audio_data,
        sr=cfg.FS,
        n_fft=cfg.N_FFT,
        hop_length=cfg.HOP_LENGTH,
        n_mels=cfg.N_MELS,
        fmin=cfg.FMIN,
        fmax=cfg.FMAX,
        power=2.0
    )

    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8)

    if CFG.apply_spec_contrast:
        mel_spec_norm = enhance_spectrogram_contrast(mel_spec_norm, CFG.contrast_factor)

    if mel_spec_norm.shape != cfg.TARGET_SHAPE:
        mel_spec_norm = cv2.resize(mel_spec_norm, cfg.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR)

    return mel_spec_norm.astype(np.float32)

def enhance_spectrogram_contrast(spec, factor=0.15):
    mean = np.mean(spec)
    enhanced = mean + (spec - mean) * (1 + factor)
    return np.clip(enhanced, 0, 1)

def apply_spec_augment(spec):
    if not CFG.use_spec_augment:
        return spec

    augmented = spec.copy()

    for _ in range(CFG.freq_mask_count):
        f = np.random.randint(0, CFG.freq_mask_param)
        f0 = np.random.randint(0, augmented.shape[0] - f)
        augmented[f0:f0 + f, :] = 0

    for _ in range(CFG.time_mask_count):
        t = np.random.randint(0, CFG.time_mask_param)
        t0 = np.random.randint(0, augmented.shape[1] - t)
        augmented[:, t0:t0 + t] = 0

    return augmented

def apply_tta(spec, tta_idx):
    result = spec.copy()

    if tta_idx == 0:
        return result
    elif tta_idx == 1:
        return np.flip(result, axis=1).copy()
    elif tta_idx == 2:
        return np.flip(result, axis=0).copy()
    elif tta_idx == 3:
        return np.flip(np.flip(result, axis=1), axis=0).copy()
    elif tta_idx == 4:
        return np.roll(result, shift=3, axis=0)
    elif tta_idx == 5:
        return np.roll(result, shift=-3, axis=0)
    else:
        return result

def predict_on_audio(audio_path, models):
    predictions = []
    row_ids = []
    soundscape_id = os.path.splitext(os.path.basename(audio_path))[0]

    try:
        print(f"Processing {soundscape_id}")
        audio_data, _ = librosa.load(audio_path, sr=CFG.FS)
        total_segments = int(len(audio_data) / (CFG.FS * CFG.WINDOW_SIZE))

        segment_predictions = []

        for segment_idx in range(total_segments):
            if time.time() > TERMINATE_TIME:
                print("Time limit reached, stopping processing early")
                return row_ids, predictions

            start_sample = segment_idx * CFG.FS * CFG.WINDOW_SIZE
            end_sample = start_sample + CFG.FS * CFG.WINDOW_SIZE
            segment_audio = audio_data[start_sample:end_sample]

            end_time_sec = (segment_idx + 1) * CFG.WINDOW_SIZE
            row_id = f"{soundscape_id}_{end_time_sec}"
            row_ids.append(row_id)

            try:
                if CFG.use_tta:
                    all_preds = []
                    for tta_idx in range(CFG.tta_count):
                        mel_spec = audio2melspec(segment_audio)

                        if np.random.random() < 0.5 and CFG.use_spec_augment:
                            mel_spec = apply_spec_augment(mel_spec)

                        mel_spec = apply_tta(mel_spec, tta_idx)
                        mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                        mel_spec_tensor = mel_spec_tensor.to(CFG.device)

                        segment_preds = []
                        for model in models:
                            with torch.no_grad():
                                outputs = model(mel_spec_tensor)
                                probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                                segment_preds.append(probs)

                        if len(models) > 1:
                            weights = np.linspace(0.8, 1.2, len(models))
                            weights = weights / weights.sum()
                            avg_preds = np.average(segment_preds, axis=0, weights=weights)
                        else:
                            avg_preds = segment_preds[0]

                        all_preds.append(avg_preds)

                    final_preds = np.mean(all_preds, axis=0)
                else:
                    mel_spec = audio2melspec(segment_audio)
                    mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                    mel_spec_tensor = mel_spec_tensor.to(CFG.device)

                    segment_preds = []
                    for model in models:
                        with torch.no_grad():
                            outputs = model(mel_spec_tensor)
                            probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                            segment_preds.append(probs)

                    if len(models) > 1:
                        weights = np.linspace(0.8, 1.2, len(models))
                        weights = weights / weights.sum()
                        final_preds = np.average(segment_preds, axis=0, weights=weights)
                    else:
                        final_preds = segment_preds[0]
            except Exception as e:
                print(f"Error processing segment {segment_idx}: {e}")
                final_preds = np.zeros(len(primary_labels)) if len(predictions) == 0 else predictions[-1]

            segment_predictions.append(final_preds)

        for i in range(len(segment_predictions)):
            if i > 0 and i < len(segment_predictions) - 1:
                blended_pred = (
                    CFG.prediction_blend[0] * segment_predictions[i] +
                    CFG.prediction_blend[1] * 0.5 * (segment_predictions[i-1] + segment_predictions[i+1])
                )
                predictions.append(blended_pred)
            else:
                predictions.append(segment_predictions[i])

    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        row_ids = []
        predictions = []

    return row_ids, predictions

def smooth_predictions(submission_df):
    print("Smoothing prediction results...")
    sub = submission_df.copy()
    cols = sub.columns[1:]
    sub['group'] = sub['row_id'].str.rsplit('_', n=1).str[0]
    unique_groups = sub['group'].unique()

    for group in unique_groups:
        group_mask = sub['group'] == group
        sub_group = sub[group_mask].copy()
        predictions = sub_group[cols].values
        new_predictions = predictions.copy()

        if predictions.shape[0] > 1:
            window = CFG.smoothing_window
            weights = CFG.smoothing_weights
            half_window = window // 2

            for i in range(half_window):
                valid_window = i + half_window + 1
                valid_weights = weights[-valid_window:]
                valid_weights = valid_weights / np.sum(valid_weights)
                new_predictions[i] = np.average(predictions[:valid_window], axis=0, weights=valid_weights)

                valid_window = i + half_window + 1
                valid_weights = weights[:valid_window]
                valid_weights = valid_weights / np.sum(valid_weights)
                new_predictions[-(i+1)] = np.average(predictions[-valid_window:], axis=0, weights=valid_weights)

            for i in range(half_window, predictions.shape[0] - half_window):
                window_start = i - half_window
                window_end = i + half_window + 1
                new_predictions[i] = np.average(predictions[window_start:window_end], axis=0, weights=weights)

        sub.loc[group_mask, cols] = new_predictions

    sub.drop('group', axis=1, inplace=True)
    return sub

def apply_thresholds(submission_df, thresholds):
    print("Applying class-specific thresholds...")
    result_df = submission_df.copy()

    for species in primary_labels:
        threshold = thresholds.get(species, CFG.threshold)
        result_df[species] = (result_df[species] >= threshold).astype(float)

    return result_df

def run_pipeline():
    print(f"Device: {CFG.device}")
    print(f"TTA enabled: {CFG.use_tta} (variations: {CFG.tta_count if CFG.use_tta else 0})")

    if CFG.class_thresholds is None:
        CFG.class_thresholds = estimate_class_thresholds()

    models = []
    for path_idx, model_path in enumerate(CFG.model_path_list):
        print(f"\nAttempting to load models from path {path_idx+1}/{len(CFG.model_path_list)}: {model_path}")
        CFG.model_path = model_path
        models = load_models()
        if models:
            print(f"Successfully loaded {len(models)} models from {model_path}")
            break
        else:
            print(f"No models loaded from {model_path}, trying next path...")

    if not models:
        print("No models found after trying all paths! Please check model paths.")
        return None

    print(f"Model usage: {'Single model' if len(models) == 1 else f'Ensemble of {len(models)} models'}")

    audio_paths, file_ids = get_audio_files()
    print(f"Found {len(audio_paths)} audio files")

    all_row_ids = []
    all_predictions = []

    for i, audio_path in enumerate(tqdm(audio_paths)):
        row_ids, predictions = predict_on_audio(audio_path, models)

        if len(row_ids) > 0 and len(predictions) > 0 and len(row_ids) == len(predictions):
            all_row_ids.extend(row_ids)
            all_predictions.extend(predictions)
        else:
            print(f"Skipping results for {audio_path} due to length mismatch or empty results")

        if (i + 1) % CFG.clear_cache_frequency == 0:
            clear_memory()

    print("Creating submission dataframe...")
    submission_dict = {'row_id': all_row_ids}
    for i, species in enumerate(primary_labels):
        submission_dict[species] = [pred[i] for pred in all_predictions]

    lengths = [len(v) for v in submission_dict.values()]
    if len(set(lengths)) > 1:
        print(f"Warning: Inconsistent lengths in submission_dict: {lengths}")

        min_length = min(lengths)
        for k in submission_dict:
            submission_dict[k] = submission_dict[k][:min_length]

    submission_df = pd.DataFrame(submission_dict)
    sample_sub = pd.read_csv(CFG.submission_csv)
    missing_cols = set(sample_sub.columns) - set(submission_df.columns)
    if missing_cols:
        print(f"Warning: Missing {len(missing_cols)} columns in submission")
        for col in missing_cols:
            submission_df[col] = 0.0

    if 'row_id' in sample_sub.columns:
        submission_df = submission_df[sample_sub.columns]

    if CFG.apply_smoothing:
        submission_df = smooth_predictions(submission_df)

    if CFG.apply_secondary_smoothing:
        cols = submission_df.columns[1:]
        groups = submission_df['row_id'].str.rsplit('_', n=1).str[0].values

        for group in np.unique(groups):
            group_mask = (groups == group)
            sub_group = submission_df[group_mask]
            predictions = sub_group[cols].values
            new_predictions = predictions.copy()

            for i in range(1, predictions.shape[0]-1):
                new_predictions[i] = (predictions[i-1] * 0.2) + (predictions[i] * 0.6) + (predictions[i+1] * 0.2)

            if predictions.shape[0] > 1:
                new_predictions[0] = (predictions[0] * 0.8) + (predictions[1] * 0.2)
                new_predictions[-1] = (predictions[-1] * 0.8) + (predictions[-2] * 0.2)

            submission_df.loc[group_mask, cols] = new_predictions

    final_df = submission_df

    submission_path = 'submission.csv'
    final_df.to_csv(submission_path, index=False)
    print(f"Submission saved to {submission_path}")

    print("\nSubmission head:")
    print(final_df.head(5))

    print("\nSubmission tail:")
    print(final_df.tail(5))

    return final_df

def create_mel_transform():
    return AT.MelSpectrogram(
        sample_rate=CFG.FS,
        n_fft=CFG.N_FFT,
        win_length=CFG.N_FFT,
        hop_length=CFG.HOP_LENGTH,
        center=True,
        f_min=CFG.FMIN,
        f_max=CFG.FMAX,
        pad_mode="reflect",
        power=2.0,
        norm='slaney',
        n_mels=CFG.N_MELS,
        mel_scale="htk",
    )

def audio_to_mel_debug(filepath):
    waveform, _ = torchaudio.load(filepath, backend="soundfile")
    waveform = waveform / torch.max(torch.abs(waveform))
    mel_transform = create_mel_transform()
    melspec = mel_transform(waveform)
    melspec = 10 * torch.log10(melspec + 1e-10)
    return melspec

def plot_results(results, file_name, audio_dir):
    path = os.path.join(audio_dir, file_name + ".ogg")
    specgram = audio_to_mel_debug(path)

    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    axes[0].set_title(file_name)
    im = axes[0].imshow(specgram[0], origin="lower", aspect="auto")
    axes[0].set_ylabel("mel bin")
    axes[0].set_xlabel("frame")
    fig.colorbar(im, ax=axes[0])

    file_results = results[results["row_id"].str.contains(file_name)]

    time_segments = file_results.shape[0]

    heatmap = axes[1].pcolor(file_results.iloc[:, 1:].values.T, edgecolors='k',
                             linewidths=0.1, vmin=0, vmax=1, cmap='Blues')
    fig.colorbar(heatmap, ax=axes[1])

    axes[1].set_xticks(np.arange(0, time_segments, 1))
    axes[1].set_xticklabels(np.arange(0, time_segments * 5, 5))

    axes[1].set_ylabel("species")
    axes[1].set_xlabel("sec")

    if len(primary_labels) <= 30:
        axes[1].set_yticks(np.arange(0.5, len(primary_labels), 1))
        axes[1].set_yticklabels(primary_labels)

    fig.tight_layout()
    plt.savefig(f'{file_name}_prediction.png')
    plt.close()

def check_paths_exist():

    for path_name, path in [
        ("test_soundscapes", CFG.test_soundscapes),
        ("train_soundscapes", CFG.train_soundscapes),
        ("submission_csv", CFG.submission_csv),
        ("taxonomy_csv", CFG.taxonomy_csv)
    ]:
        exists = os.path.exists(path)
        print(f"{path_name}: {path} - {'exist' if exists else 'not exist'}")

    for i, path in enumerate(CFG.model_path_list):
        exists = os.path.exists(path)
        print(f"model_path_list[{i}]: {path} - {'exist' if exists else 'not exist'}")


if __name__ == "__main__":
    try:
        check_paths_exist()

        results = run_pipeline()

        clear_memory()

        if CFG.debug and results is not None:
            audio_paths, file_ids = get_audio_files()
            audio_dir = CFG.train_soundscapes if CFG.debug else CFG.test_soundscapes

            print("\nGenerating visualizations...")
            for file_id in file_ids:
                plot_results(results, file_id, audio_dir)
                print(f"Visualization saved for {file_id}")

        print(f"\nTotal execution time: {(time.time() - START) / 60:.2f} minutes")
    except Exception as e:
        print(f"Error during execution: {e}")
        import traceback
        traceback.print_exc()